In [7]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
from sklearn.metrics import adjusted_rand_score

In [11]:
# Buscar dataset com ID 17
dataset = fetch_ucirepo(id=537)

# Converter os dados em DataFrame
df = pd.DataFrame(dataset.data.features)

# Se quiser adicionar os rótulos (se existirem)
if dataset.data.targets is not None:
    # targets pode ser Series ou DataFrame dependendo do conjunto
    targets = pd.DataFrame(dataset.data.targets)
    df = pd.concat([df, targets], axis=1)

# Visualizar os dados
df.head()

,behavior_eating,behavior_personalHygiene,intention_aggregation,intention_commitment,attitude_consistency,attitude_spontaneity,norm_significantPerson,norm_fulfillment,perception_vulnerability,perception_severity,motivation_strength,motivation_willingness,socialSupport_emotionality,socialSupport_appreciation,socialSupport_instrumental,empowerment_knowledge,empowerment_abilities,empowerment_desires,behavior_sexualRisk,ca_cervix
0,13,12,4,7,9,10,1,8,7,3,14,8,5,7,12,12,11,8,10,1
1,11,11,10,14,7,7,5,5,4,2,15,13,7,6,5,5,4,4,10,1
2,15,3,2,14,8,10,1,4,7,2,7,3,3,6,11,3,3,15,10,1
3,11,10,10,15,7,7,1,5,4,2,15,13,7,4,4,4,4,4,10,1
4,11,7,8,10,7,8,1,5,3,2,15,5,3,6,12,5,4,7,8,1


In [9]:
labels = df["ca_cervix"].values
df.drop("ca_cervix", axis=1, inplace=True)
dados = df.to_numpy()

In [13]:
labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [10]:
def inicializacao_matriz_pertinencia(num_amostras, num_clusters):
    matriz_pertinencia = np.random.rand(num_amostras, num_clusters) # gera uma matriz inicial aleatória com valores entre 0 e 1
    matriz_pertinencia = matriz_pertinencia / matriz_pertinencia.sum(axis=1, keepdims=True) # normalização da matriz pra garantir que a soma dos graus dê um
    return matriz_pertinencia

# %% [markdown]
# ### Atualização dos centroides

# %% [markdown]
# Fixo os graus de pertinência, os centroides são atualizados com base nessa equação:
# 
# ### $y_i = \frac{\sum_{k=1}^n(u_{ik})^mx_k}{\sum_{k=1}^n(u_{ik})^m}$

# %%
def atualizacao_centroides(dados, matriz_pertinencia, m):
    matriz_pertinencia_m = matriz_pertinencia ** m # preparação dos graus de pertinência
    centroides = np.dot(matriz_pertinencia_m.T, dados) / np.sum(matriz_pertinencia_m.T, axis=1, keepdims=True) # fórmula para o cálculo dos centroides
    return centroides

# %% [markdown]
# ### Atualização da matriz de pertinência

# %% [markdown]
# Fixo o protótipo, os graus de pertinência são atualizados com base nessa equação:
# 
# ### $u_{ik} = [\sum_{h=1}^c\{\frac{d(x_k,y_i)}{d(x_k,y_h)}\}^\frac{1}{m-1}]^{-1}$
# 
# onde
# 
# $d(x_k,y_i) = \sum_{j=1}^p(x_k^j-y_i^j)^2$

# %%
def atualizacao_matriz_pertinencia(dados, centroides, m):
    # dados[:, np.newaxis] - centroides cria uma matriz de diferenças entre os pontos de dados e os centroides
    # np.linalg.norm(..., axis=2) calcula a norma (distância euclidiana) das diferenças
    # ** 2 para a distância ser a quadrada
    matriz_distancias = np.linalg.norm(dados[:, np.newaxis] - centroides, axis=2) ** 2
    matriz_distancias = np.fmax(matriz_distancias, np.finfo(np.float64).eps) # evita que matriz_distancias seja 0, np.finfo... é o menor número maior que zero aaqui
    matriz_distancias_inversa = 1 / matriz_distancias
    potencia = 1 / (m-1)
    matriz_pertinencia_atualizada = matriz_distancias_inversa ** potencia/ np.sum(matriz_distancias_inversa ** potencia, axis=1, keepdims=True) # fórmula para atualizar os graus de pertinência
    return matriz_pertinencia_atualizada

# %% [markdown]
# ### Fuzzy C-Means

# %% [markdown]
# Ações:
# 1. Inicialização da matriz de pertinência
# 2. Atualização dos centroides
# 3. Atualização da matriz de pertinência
# 
# Critérios de parada:
# 1. Número máximo de iterações atingido
# 2. Pouca diferença (erro) entre as matrizes de pertinência de iterações consecutivas

# %%
def fcm(dados, num_clusters, m=2, max_iter=10**6, erro=1e-9):
    num_amostras = dados.shape[0]
    matriz_pertinencia = inicializacao_matriz_pertinencia(num_amostras, num_clusters)
    for _ in range(max_iter): # primeiro critério de parada
        centroides = atualizacao_centroides(dados, matriz_pertinencia, m)
        nova_matriz_pertinencia = atualizacao_matriz_pertinencia(dados, centroides, m)
        if np.linalg.norm(nova_matriz_pertinencia - matriz_pertinencia) < erro: # segundo critério de parada
            break
        matriz_pertinencia = nova_matriz_pertinencia
    return centroides, matriz_pertinencia

# %% [markdown]
# ### Índice de Rand Ajustado (IRA)

# %%
def indice_rand(labels, predicted_labels):
    return adjusted_rand_score(labels, predicted_labels)

# %% [markdown]
# ### Simulação de Monte Carlo

# %%
def simulacao_monte_carlo(dados, labels, num_clusters, num_trials):
    indices_rand = []
    for _ in range(num_trials):
        centroides, matriz_pertinencia = fcm(dados, num_clusters)
        predicted_labels = np.argmax(matriz_pertinencia, axis=1)
        #print(predicted_labels)
        idx_rand = indice_rand(labels, predicted_labels)
        indices_rand.append(idx_rand)
    mean_rand_index = np.mean(indices_rand)
    std_rand_index = np.std(indices_rand)
    return mean_rand_index, std_rand_index

# %% [markdown]
# ### Definição de parâmetros e execução do método

# %%
num_clusters = 2
num_trials = 100
media_indice_rand, dp_indice_rand = simulacao_monte_carlo(dados, labels, num_clusters, num_trials)

print(f"Monte Carlo FCM Clustering Results ({num_trials} trials)")
print(f"Mean Rand Index: {media_indice_rand:.4f}")
print(f"Standard Deviation of Rand Index: {dp_indice_rand:.4f}")

Monte Carlo FCM Clustering Results (100 trials)
Mean Rand Index: 0.2667
Standard Deviation of Rand Index: 0.0000
